#### Update (if data exist) or append (if data doesn't exist) df_finviz_merged_stocks_etfs.describe() to STATS_HISTORY_FILE

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged_stocks_etfs.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_data: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']
c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-25_df_finviz_merged_stocks_etfs.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 2000)        # Let the display adjust to the window0)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# # liquidity filter, Avg Volume, M > 0.75M
# df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# # Drop specified columns with NaNs in df_data
# df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# print(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# print(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
print(df_data.head())
print('\n')
print((df_data.describe()))


df_data.shape: (1518, 139)
        No.                Company               Index                  Sector                        Industry Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M     P/E  Fwd P/E    PEG     P/S     P/B     P/C   P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %     EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %    Sales, M   Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M   Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M   ROA %    ROE %   ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %   Beta     ATR  ATR/Price %  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %

In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [ ]:
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-25
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv
Updating existing entry for 2025-04-25
Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
print(df_data.describe())

date_str: 2025-04-25
            No.  MktCap AUM, M      Rank  Market Cap, M        P/E   Fwd P/E      PEG       P/S       P/B        P/C      P/FCF     Book/sh     Cash/sh  Dividend %  Dividend TTM  Payout Ratio %        EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %   EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %    Sales, M   Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M   Float, M   Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M     ROA %     ROE %     ROI %   Curr R  Quick R  LTDebt/Eq   Debt/Eq  Gross M %   Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %      Beta        ATR  ATR/Price %  Volatility W %  Volatility M %   SMA20 %   SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %  All-Time High %  All-Time Low %       RSI    Employees  Change from Open %     Gap %     Rec

In [8]:
print(f'date_str: {date_str}')
print(df_data.head())

date_str: 2025-04-25
        No.                Company               Index                  Sector                        Industry Country Exchange                                               Info  MktCap AUM, M  Rank  Market Cap, M     P/E  Fwd P/E    PEG     P/S     P/B     P/C   P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %     EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %    Sales, M   Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M   Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M   ROA %    ROE %   ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf 3D %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %   Beta     ATR  ATR/Price %  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D H